# BT01: Thu thập dữ liệu từ web

Họ tên: Trương Ngọc Tài

MSSV: 1512473

---

## <font color=blue>Tổng thể</font>

**Làm bài**

Bạn sẽ làm trực tiếp trên file notebook này. Đầu tiên, bạn điền họ tên và MSSV vào phần đầu file ở bên trên. Trong file, mình đã để từ `TODO` để cho biết những chỗ mà bạn cần phải làm. Trong khi làm bài, thường xuyên `Ctrl + S` để lưu lại bài làm của bạn, tránh mất mát thông tin.

*Nên nhớ mục tiêu chính ở đây là <font color=green>học, học một cách chân thật</font>. Bạn có thể thảo luận ý tưởng với bạn khác, nhưng <font color=green>code và bài làm phải là của bạn, dựa trên sự hiểu thật sự của bạn</font>. <font color=red>Nếu vi phạm thì sẽ bị 0 điểm cho toàn bộ môn học.</font>*

**Nộp bài**

Khi chấm bài, đầu tiên mình sẽ chọn `Kernel` - `Restart & Run All` để restart và chạy tất cả các cell trong notebook của bạn; do đó, trước khi nộp bài, bạn nên chạy thử `Kernel` - `Restart & Run All` để đảm bảo mọi chuyện diễn ra đúng như mong đợi.

Sau đó, trong thư mục `MSSV` (vd, nếu bạn có MSSV là 1234567 thì bạn đặt tên thư mục là `1234567`) bạn đặt file `BT01-ThuThapDuLieuWeb.ipynb`, rồi nén thư mục `MSSV` này lại và nộp ở link trên moodle.

**Nội dung bài tập**

Trong bài này, bạn sẽ hoàn thành ví dụ tình hình tuyển dụng công việc về Khoa Học Dữ Liệu mà mình đang làm dở ở trên lớp lý thuyết.

## <font color=blue>Đặt câu hỏi</font>
**Tình hình tuyển dụng các công việc về khoa học dữ liệu ở Việt Nam hiện nay như thế nào?**

Ban đầu, câu hỏi đặt ra thường rộng và mơ hồ. Lúc sau, sẽ quay lại bước này để điều chỉnh câu hỏi cho rõ ràng và cụ thể hơn.

## <font color=blue>Thu thập dữ liệu</font>

In [74]:
import requests
from bs4 import BeautifulSoup

In [75]:
# Read html text
url = 'https://careerbuilder.vn/viec-lam/data-science-k-vi.html'
html_text = requests.get(url).text
# html_text

In [76]:
# Parse html text
tree = BeautifulSoup(html_text, 'html.parser')

In [77]:
# Find all job containers
job_containers = tree.find_all('dd')
len(job_containers)

22

Kế đến, với mỗi phần tử (mỗi tuyển dụng) trong `job_containers`, ta sẽ rút trích ra các thông tin:
- Công ty tuyển dụng (đã làm trên lớp lý thuyết).
- Địa điểm (đã làm trên lớp lý thuyết).
- Ngày đăng thông báo (đã làm trên lớp lý thuyết).
- Lương.
- Tựa đề.
- Link đến nội dung chi tiết.
- Nội dung chi tiết (chỗ này có điều chỉnh so với trên lớp lý thuyết, vì mình nghĩ lại và thấy rằng ban đầu khi chưa rõ về câu hỏi đặt ra thì có lẽ nên thu thập thông tin đầy đủ và nguyên thủy một chút). Ở phía dưới, mình đã cung cấp sẵn cho bạn hàm `extract_text` - hàm rút trích nội dung văn bản từ địa chỉ của một trang web.

Sau đó, lưu các thông tin này xuống file `csv` (mỗi tuyển dụng ứng với một dòng). Để tránh nhập nhằng giữa dấu `,` trong nội dung các thông tin và dấu `,` phân tách các giá trị trong file `csv`, ta sẽ dùng dấu `\t` (tab) để phân tách các giá trị trong file `csv` (file `csv` cho phép làm như vậy, chứ không bắt buộc là phải dùng dấu `,`).

In [78]:
def extract_text(url):
    '''
    Extracts text from a web.
    
    Parameters
    ----------
    url : string
        The url of a web.
    
    Returns
    -------
    text : string
        The extracted text.
    '''
    # Read html text
    try:
        html_text = requests.get(url).text
    except:
        print('Cannot read ' + url)
        return ''
    
    # Parse html text
    tree = BeautifulSoup(html_text, 'html.parser')
    
    # Remove invisible tags from tree
    invisible_tags = tree.find_all(['style', 'script'])
    for invisible_tag in invisible_tags:
        invisible_tag.extract()
    
    # Get text from tree, replace new line character by ' ' to prevent problem when writing to csv file 
    text = ' '.join(tree.stripped_strings).replace('\r\n', ' ').replace('\n', ' ')
    
    return text
# text = extract_word_set('https://careerbuilder.vn/vi/tim-viec-lam/tan-binh-ai-engineer-data-science-minimum-1700usd-l19.35B022BD.html')
# print(text)

In [79]:
f = open('ds_jobs.csv', 'w', encoding='utf-16')
f.write('company\tlocation\tdate\tsalary\ttitle\turl\tfull_text\n') # Write header to csv file

# With each job container ...
for job_container in job_containers:
    # Extract data
    company = job_container.find('p', {'class': 'namecom'}).find('a').string
    location = job_container.find('p', {'class': 'location'}).get_text()
    date = job_container.find('div', {'class': 'dateposted'}).string.replace('Ngày cập nhật: ', '')
    salary = job_container.find('p', {'class': 'salary'}).get_text().replace('Lương: ', '')
    title = job_container.find('h3', {'class': 'job'}).get_text()
    url = job_container.find('p', {'class': 'namecom'}).a.get('href')
    inf_job_url = job_container.find('h3', {'class': 'job'}).a.get('href')
    full_text = extract_text(inf_job_url)
    
    # Write to csv file
    f.write(company + '\t')
    f.write(location + '\t')
    f.write(date + '\t')
    f.write(salary + '\t')
    f.write(title + '\t')
    f.write(url + '\t')
    f.write(full_text + '\n')
    print(company)
    print(location)
    print(date)
    print(salary)
    print(title)
    print(url)
    print()

f.close()

Công Ty TNHH MTV Wacontre
Hồ Chí Minh
11/09/2018
37,4 Tr - 88 Tr VND
[Tân Bình] AI ENGINEER / DATA SCIENCE - Minimum 1700USD (L19) 
https://careerbuilder.vn/vi/nha-tuyen-dung/cong-ty-tnhh-mtv-wacontre.35A76071.html

Gigatum Vietnam JSC
Hà Nội, Hồ Chí Minh
16/09/2018
Cạnh tranh
Lead Data Scientist 
https://careerbuilder.vn/vi/nha-tuyen-dung/gigatum-vietnam-jsc.35A6C784.html

Home Credit Vietnam
Hồ Chí Minh
12/09/2018
Cạnh tranh
Future Analyst Program 
https://careerbuilder.vn/vi/nha-tuyen-dung/home-credit-vietnam.35A51A1A.html

Giao hàng tiết kiệm
Hà Nội
17/09/2018
22 Tr - 33 Tr VND
[HN] Data Scientist (1000 - 1500 USD) 
https://careerbuilder.vn/vi/nha-tuyen-dung/giao-hang-tiet-kiem.35A7B370.html

Công Ty CP Dược Phẩm Pharmacity
Hồ Chí Minh
12/09/2018
13 Tr - 17 Tr VND
Data Scientist (BI Team) 
https://careerbuilder.vn/vi/nha-tuyen-dung/cong-ty-cp-duoc-pham-pharmacity.35A7E5C8.html

FPT Software
Hokkaido, Osaka, Tokyo
12/09/2018
Cạnh tranh
[FJP] 5 AI Engineers 
https://careerbuilder.vn/

## <font color=blue>Xem dữ liệu và đặt lại câu hỏi</font>
Bây giờ, hãy mở file `ds_jobs.csv` để xem dữ liệu: double click vào file `ds_jobs.csv` và file này sẽ được mở bằng Excel (ở máy mình, mình xem file này một cách bình thường, không có vấn đề gì; nếu bạn gặp vấn đề gì thì có thể post lên moodle). Với dữ liệu như này ở trước mặt, ta hãy nghĩ về câu hỏi ban đầu và xem nên điều chỉnh lại như thế nào cho rõ ràng và cụ thể hơn? Hay là có các câu hỏi nào khác mà có thể được trả lời thông qua dữ liệu này không? Câu hỏi có hay không, nếu trả lời được câu hỏi thì sẽ có ích lợi gì không?

Một câu hỏi có thể đặt ra là: **các tuyển dụng về Khoa Học Dữ Liệu ở Việt Nam hiện nay thường yêu cầu biết ngôn ngữ lập trình nào?**  Ở đây, để đơn giản, ta sẽ giới hạn tập ngôn ngữ được khảo sát là: python, r, matlab, java, c#, c++.
- Nếu trả lời được câu hỏi này thì sẽ giúp ta định hướng nên học các ngôn ngữ lập trình nào nếu muốn theo Khoa Học Dữ Liệu.
- Câu hỏi này có thể được trả lời với tập dữ liệu trên bằng cách: với mỗi tuyển dụng, xem trong `full_text` yêu cầu các ngôn ngữ lập trình nào và cập nhật biến đếm cho các ngôn ngữ lập trình đó;  cuối cùng, khi duyệt hết tất cả các tuyển dụng, ta sẽ có được thống kê ngôn ngữ nào được yêu cầu nhiều nhất, ngôn ngữ nào được yêu cầu nhiều nhì, ... (bạn cũng nên thắc mắc: với câu trả lời có được từ tập dữ liệu được thu thập như trên thì có đáng tin cậy không, làm sao để cải thiện độ tin cậy?)

## <font color=blue>Phân tích dữ liệu</font>
Bây giờ, ta sẽ tiến hành phân tích dữ liệu để trả lời cho câu hỏi ở trên. Một cách đơn giản là: với mỗi tuyển dụng, ta sẽ rút trích ra tập các từ từ `full_text`; rồi với mỗi ngôn ngữ lập trình, kiểm xem có trong tập từ này không, nếu có thì cập nhật biến đếm tương ứng.

In [80]:
import re # Regular expression

In [81]:
f = open('ds_jobs.csv', 'r', encoding='utf-16')
lines = f.readlines()
f.close()

language_counts = {'c++': 0, 'c#': 0, 'java': 0, 'matlab': 0, 'python': 0, 'r': 0} # Init
for line in lines[1:]: # Skip the first line (header)
    full_text = line.split('\t')[-1].lower()
    word_set = set(re.findall('[\\w#+]+', full_text, re.UNICODE))
    # TODO: with each language, check if it's in word_set 
    #       if true then update the corresponding count
    for word in word_set:
        if word in language_counts.keys():
            language_counts[word] = language_counts[word] + 1
print("Ket qua cua tai dep trai la: ")    
language_counts

Ket qua cua tai dep trai la: 


{'c++': 6, 'c#': 0, 'java': 4, 'matlab': 0, 'python': 17, 'r': 10}

Kết quả chạy của mình: `{'c#': 0, 'c++': 5, 'java': 4, 'matlab': 0, 'python': 16, 'r': 10}`. Tại thời điểm bạn chạy, có thể sẽ có khác đôi chút.